In [10]:
import math
import pandas as pd
import numpy as np
from prophet import Prophet

In [11]:
df = pd.read_csv("../Data/RawTrafficData.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df_prophet = df.reset_index().rename(columns={'index': 'ds', 'value': 'y'})

train = df_prophet[:800]
test = df_prophet[801:]

In [12]:
%%timeit -r 2 -n 100
model = Prophet(interval_width=0.99)
model.fit(train)

forecastlen = len(test)
future = model.make_future_dataframe(periods=forecastlen,freq ='h')
forecast = model.predict(future)

test_data_forecasts = np.array(forecast["yhat"][800:])
ground_truth = test.y

errors = np.abs(ground_truth-test_data_forecasts)

ground_truth_values = test.y
anomaly_scores = np.zeros(len(ground_truth_values))

for i, true_value in enumerate(ground_truth_values):
    lower_bound = forecast["yhat_lower"][i]
    upper_bound = forecast["yhat_upper"][i] 
    prediction = forecast["yhat"][i]
    
    threshold = np.abs(prediction-upper_bound) * 6
    error = np.abs(true_value - prediction)

    if error >= threshold:
        anomaly_scores[i] = 1.0
    else:
        anomaly_scores[i] = error / threshold

10:57:24 - cmdstanpy - INFO - Chain [1] start processing
10:57:24 - cmdstanpy - INFO - Chain [1] done processing
10:57:24 - cmdstanpy - INFO - Chain [1] start processing
10:57:25 - cmdstanpy - INFO - Chain [1] done processing
10:57:25 - cmdstanpy - INFO - Chain [1] start processing
10:57:25 - cmdstanpy - INFO - Chain [1] done processing
10:57:25 - cmdstanpy - INFO - Chain [1] start processing
10:57:25 - cmdstanpy - INFO - Chain [1] done processing
10:57:26 - cmdstanpy - INFO - Chain [1] start processing
10:57:26 - cmdstanpy - INFO - Chain [1] done processing
10:57:26 - cmdstanpy - INFO - Chain [1] start processing
10:57:26 - cmdstanpy - INFO - Chain [1] done processing
10:57:27 - cmdstanpy - INFO - Chain [1] start processing
10:57:27 - cmdstanpy - INFO - Chain [1] done processing
10:57:27 - cmdstanpy - INFO - Chain [1] start processing
10:57:27 - cmdstanpy - INFO - Chain [1] done processing
10:57:28 - cmdstanpy - INFO - Chain [1] start processing
10:57:28 - cmdstanpy - INFO - Chain [1]

466 ms ± 4.54 ms per loop (mean ± std. dev. of 2 runs, 500 loops each)
